In [1]:
from io import StringIO
import pandas as pd
import numpy as np
from xml.etree import cElementTree as ET

In [2]:
# # THIS OFTEN FAILS BECAUSE THE SERVER IS UNRELIABLE
# # The api response is stored as 'response5.xml' instead.

# # credentials
# user = 'DERY65931W'
# password = '6zH1b%255aw77'

# # Request data to be prepared
# request1_url = "https://www-genesis.destatis.de/genesisWS/web/ExportService_2010"
# request1_data = {
#     'method': 'TabellenExport',
#     'kennung': user,
#     'passwort': password,
#     'namen': '61111-0004',
#     'bereich': 'Alle',
#     'format': 'csv',
#     'strukturinformationen': False,
#     'komprimieren': False,
#     'transponieren': True,
#     'startjahr': 1991,
#     'endjahr': 2022,
#     'auftrag': True,
#     'sprache': 'de'
# }

# encodeddata = urllib.parse.urlencode(request1_data).encode('UTF-8')
# req = urllib.request.Request(request1_url, encodeddata)
# resp1 = urllib.request.urlopen(req)

# request1_url = f'https://www-genesis.destatis.de/genesisWS/web/ExportService_2010?' \
#                'method=TabellenExport&kennung={user}&passwort={password}&' \
#               f'namen=61111-0004&bereich=Alle&format=csv&' \
#                'strukturinformation=false&komprimieren=false&' \
#                'transponieren=true&' \
#                'startjahr=1991&endjahr=2022' \
#                '&zeitscheiben=&regionalmerkmal=&regionalschluessel=&sachmerkmal=&sachschluessel=&sachmerkmal2=&sachschluessel2=&sachmerkmal3=&sachschluessel3=&stand=&auftrag=true&sprache=de'
# resp1 = urlopen(request1_url)
# resp1

In [4]:
root = ET.parse('raw_data/61111-0004.xml')

table_data = root.findall('.//tabellenDaten')
assert len(table_data) == 1     # API should only return a single element of this type
table_data = table_data[0]
csv_data = table_data.text

In [5]:
df = pd.read_csv(
    StringIO(csv_data),
    sep=';',
    decimal=',',
    skiprows=8,
    skipfooter=3
    )

df

/var/folders/ng/wqc4d0qd0b92x6j5y4_0xzg80000gn/T/ipykernel_5523/4099931074.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(


,Unnamed: 0,Unnamed: 1,CC13-01 Nahrungsmittel und alkoholfreie Getränke,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 429,Unnamed: 430,Unnamed: 431,Unnamed: 432,Unnamed: 433,Unnamed: 434,Unnamed: 435,Unnamed: 436,Unnamed: 437,Unnamed: 438
0,NaN,NaN,Insgesamt,CC13-011 Nahrungsmittel,NaN,NaN,NaN,NaN,NaN,NaN,...,"CC13-126 Finanzdienstleistungen, a.n.g.",NaN,NaN,NaN,"CC13-127 Andere Dienstleistungen, a.n.g.",NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,Insgesamt,CC13-0111 Brot und Getreideerzeugnisse,NaN,NaN,NaN,NaN,NaN,...,Insgesamt,"CC13-1262 Andere Finanzdienstleistungen, a.n.g.",NaN,NaN,Insgesamt,"CC13-1270 Andere Dienstleistungen, a.n.g.",NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,Insgesamt,"CC13-01111 Reis, einschließlich Reiszubereitungen",CC13-01112 Mehl und andere Getreideerzeugnisse,CC13-01113 Brot und Brötchen,CC13-01114 Andere Backwaren,"CC13-01115 Pizza, Quiches und Ähnliches",...,NaN,Insgesamt,CC13-12621 Bank- und Sparkassengebühren und Äh...,"CC13-12622 Gebühren für Anlageberatung, Steuer...",NaN,Insgesamt,CC13-12701 Verwaltungsgebühren,"CC13-12702 Rechtsberatung, Rechtsanwalts- und ...",CC13-12703 Bestattungsdienstleistungen,CC13-12704 Andere Gebühren und Dienstleistungen
3,1991.0,Januar,70.0,69.5,64.5,.,.,.,.,.,...,60.9,60.9,.,.,53.4,53.4,.,.,.,.
4,NaN,Februar,70.4,70.0,64.6,.,.,.,.,.,...,60.9,60.9,.,.,53.4,53.4,.,.,.,.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
382,NaN,August,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383,NaN,September,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
384,NaN,Oktober,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,NaN,November,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [6]:
df.rename(
    columns={
        df.columns[0]: "year",
        df.columns[1]: "month"
    },
    inplace=True)

# forward-propagate the year values to fill NaN
df['year'] = df['year'].fillna(method='ffill')

# we only want to keep the top-level classes, discard all other data

# remove rows containing headers for lower-level classes
df = df[3:]

# remove columns containing data for lower-level classes
columns_to_drop = [
    column
    for column in df.columns
    if column.startswith('Unnamed')]
df.drop(columns_to_drop, axis=1, inplace=True)

# convert year column to integer type
df['year'] = df['year'].astype(int)

# convert months to numerical values
df['month'] = df['month'].replace({
    'Januar': 1,
    'Februar': 2,
    'März': 3,
    'April': 4,
    'Mai': 5,
    'Juni': 6,
    'Juli': 7,
    'August': 8,
    'September': 9,
    'Oktober': 10,
    'November': 11,
    'Dezember': 12
}).astype(int)

# convert the columns year and month into a date-type column
df.loc[:, 'day'] = 15
df.loc[:, 'date'] = pd.to_datetime(df[['year', 'month', 'day']])
df.drop(['year', 'month', 'day'], axis=1, inplace=True)

# use the date column as index
df.set_index('date', inplace=True)

# convert data from string to float
df.replace('...', np.nan, inplace=True)
df = df.astype(float)

# drop some empty rows
df.dropna(inplace=True)

# replacing missing values with nan
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.replace('-', '_')
df.columns = df.columns.str.replace(',', '')

# storing table as .csv
df.to_csv('61111-0004.csv', sep=',')

df

/var/folders/ng/wqc4d0qd0b92x6j5y4_0xzg80000gn/T/ipykernel_5523/2690096997.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns_to_drop, axis=1, inplace=True)
/var/folders/ng/wqc4d0qd0b92x6j5y4_0xzg80000gn/T/ipykernel_5523/2690096997.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['year'] = df['year'].astype(int)
/var/folders/ng/wqc4d0qd0b92x6j5y4_0xzg80000gn/T/ipykernel_5523/2690096997.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


,CC13_01_Nahrungsmittel_und_alkoholfreie_Getränke,CC13_02_Alkoholische_Getränke_und_Tabakwaren,CC13_03_Bekleidung_und_Schuhe,CC13_04_Wohnung_Wasser_Strom_Gas_und_andere_Brennstoffe,CC13_05_Möbel_Leuchten_Geräte_u.a._Haushaltszubehör,CC13_06_Gesundheit,CC13_07_Verkehr,CC13_08_Post_und_Telekommunikation,CC13_09_Freizeit_Unterhaltung_und_Kultur,CC13_10_Bildungswesen,CC13_11_Gaststätten__und_Beherbergungsdienstleistungen,CC13_12_Andere_Waren_und_Dienstleistungen
date,,,,,,,,,,,,
1991-01-15,70.0,50.2,83.8,53.5,81.7,59.5,54.3,157.0,80.6,42.2,60.2,59.8
1991-02-15,70.4,50.2,83.9,53.8,81.9,59.7,54.3,157.0,81.9,42.4,61.6,59.9
1991-03-15,70.8,50.3,84.2,53.3,82.2,59.8,54.5,157.0,82.1,42.8,61.1,60.0
1991-04-15,71.2,50.4,84.3,53.5,82.4,59.9,55.1,161.5,80.3,43.7,61.1,60.1
1991-05-15,71.2,50.4,84.6,54.0,82.8,59.9,55.4,161.5,80.7,44.6,62.0,60.1
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-15,116.7,118.4,100.9,112.2,107.8,106.2,119.1,94.4,103.7,105.5,117.5,112.1
2022-02-15,117.8,119.1,101.2,113.1,108.9,106.3,121.0,94.3,105.7,105.6,118.2,112.4
2022-03-15,118.8,119.7,105.8,116.6,109.4,106.7,129.9,94.2,106.5,105.7,118.8,112.8


In [8]:
# normalizing dataframe to first entry
df_std = df/df.iloc[0]*100

# storing converted dataframe in local directory
df_std.to_csv('preped_data/61111-0004_std.csv', sep=',') 
df_std

,CC13_01_Nahrungsmittel_und_alkoholfreie_Getränke,CC13_02_Alkoholische_Getränke_und_Tabakwaren,CC13_03_Bekleidung_und_Schuhe,CC13_04_Wohnung_Wasser_Strom_Gas_und_andere_Brennstoffe,CC13_05_Möbel_Leuchten_Geräte_u.a._Haushaltszubehör,CC13_06_Gesundheit,CC13_07_Verkehr,CC13_08_Post_und_Telekommunikation,CC13_09_Freizeit_Unterhaltung_und_Kultur,CC13_10_Bildungswesen,CC13_11_Gaststätten__und_Beherbergungsdienstleistungen,CC13_12_Andere_Waren_und_Dienstleistungen
date,,,,,,,,,,,,
1991-01-15,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
1991-02-15,100.571429,100.000000,100.119332,100.560748,100.244798,100.336134,100.000000,100.000000,101.612903,100.473934,102.325581,100.167224
1991-03-15,101.142857,100.199203,100.477327,99.626168,100.611995,100.504202,100.368324,100.000000,101.861042,101.421801,101.495017,100.334448
1991-04-15,101.714286,100.398406,100.596659,100.000000,100.856793,100.672269,101.473297,102.866242,99.627792,103.554502,101.495017,100.501672
1991-05-15,101.714286,100.398406,100.954654,100.934579,101.346389,100.672269,102.025783,102.866242,100.124069,105.687204,102.990033,100.501672
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-15,166.714286,235.856574,120.405728,209.719626,131.946144,178.487395,219.337017,60.127389,128.660050,250.000000,195.182724,187.458194
2022-02-15,168.285714,237.250996,120.763723,211.401869,133.292534,178.655462,222.836096,60.063694,131.141439,250.236967,196.345515,187.959866
2022-03-15,169.714286,238.446215,126.252983,217.943925,133.904529,179.327731,239.226519,60.000000,132.133995,250.473934,197.342193,188.628763


In [9]:
# calculate year-over-year inflation rate
df_pct = df.pct_change(freq=pd.DateOffset(years=1))
df_pct.dropna(inplace=True)

# calculate change in percent
df_pct = df_pct*100

# storing converted dataframe in local directory
df_pct.to_csv('preped_data/61111-0004_pct.csv', sep=',')

df_pct

,CC13_01_Nahrungsmittel_und_alkoholfreie_Getränke,CC13_02_Alkoholische_Getränke_und_Tabakwaren,CC13_03_Bekleidung_und_Schuhe,CC13_04_Wohnung_Wasser_Strom_Gas_und_andere_Brennstoffe,CC13_05_Möbel_Leuchten_Geräte_u.a._Haushaltszubehör,CC13_06_Gesundheit,CC13_07_Verkehr,CC13_08_Post_und_Telekommunikation,CC13_09_Freizeit_Unterhaltung_und_Kultur,CC13_10_Bildungswesen,CC13_11_Gaststätten__und_Beherbergungsdienstleistungen,CC13_12_Andere_Waren_und_Dienstleistungen
date,,,,,,,,,,,,
1992-01-15,3.428571,3.784861,2.625298,9.719626,2.815177,3.697479,7.182320,6.815287,3.349876,9.715640,5.149502,4.515050
1992-02-15,3.693182,4.183267,2.741359,9.851301,3.052503,3.852596,7.550645,6.815287,3.418803,11.320755,4.383117,5.008347
1992-03-15,2.824859,5.367793,2.731591,11.069418,2.919708,4.013378,8.440367,6.815287,3.166870,10.280374,4.909984,5.166667
1992-04-15,2.808989,5.357143,2.846975,11.028037,2.791262,4.173623,8.348457,3.839009,4.732254,9.153318,6.382979,5.324459
1992-05-15,3.089888,5.555556,2.718676,10.555556,2.777778,4.173623,7.942238,3.467492,3.841388,6.950673,5.161290,5.657238
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-15,4.851752,3.587052,-0.099010,5.154639,3.157895,0.854701,10.584958,-0.105820,5.386179,1.833977,4.444444,2.374429
2022-02-15,5.084746,3.745645,-1.075269,5.799813,4.011461,0.758294,11.213235,-0.105932,4.757185,1.734104,5.066667,2.181818
2022-03-15,5.882353,3.996525,2.025072,8.768657,4.688995,0.946074,17.450271,-0.106045,5.029586,1.536984,5.600000,1.989150


In [10]:
df = pd.read_excel('raw_data/61111-0002.xlsx')
df


,Year,Month,Verbraucherpreisindex,Veränderung zum Vorjahresmonat,Veränderung zum Vormonat
0,1991.0,Januar,64,.,.
1,NaN,Februar,64.3,.,0.5
2,NaN,März,64.3,.,-
3,NaN,April,64.5,.,0.3
4,NaN,Mai,64.7,.,0.3
...,...,...,...,...,...
379,NaN,August,...,...,...
380,NaN,September,...,...,...
381,NaN,Oktober,...,...,...
382,NaN,November,...,...,...


In [11]:
# forward-propagate the year values to fill NaN
df['Year'] = df['Year'].fillna(method='ffill')

# convert year column to integer type
df['Year'] = df['Year'].astype(int)

# convert months to numerical values
df['Month'] = df['Month'].replace({
    'Januar': 1,
    'Februar': 2,
    'März': 3,
    'April': 4,
    'Mai': 5,
    'Juni': 6,
    'Juli': 7,
    'August': 8,
    'September': 9,
    'Oktober': 10,
    'November': 11,
    'Dezember': 12
}).astype(int)

# convert the columns year and month into a date-type column
df.loc[:, 'day'] = 15
df.loc[:, 'date'] = pd.to_datetime(df[['Year', 'Month', 'day']])
df.drop(['Year', 'Month', 'day'], axis=1, inplace=True)

# use the date column as index
df.set_index('date', inplace=True)


# replacing missing values with nan
df.replace('...', np.nan, inplace=True)
df.replace('.', np.nan, inplace=True)
df.replace('-', np.nan, inplace=True)

# convert data from string to float
df = df.astype(float)

# drop some empty rows
df.dropna(inplace=True)

# replacing special characters in column names
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.replace('-', '_')
df.columns = df.columns.str.replace(',', '')

# storing prepared dataframe
df.to_csv('preped_data/61111-0002.csv', sep=',')

df

,Verbraucherpreisindex,Veränderung_zum_Vorjahresmonat,Veränderung_zum_Vormonat
date,,,
1992-01-15,67.6,5.6,0.6
1992-02-15,68.0,5.8,0.6
1992-03-15,68.3,6.2,0.4
1992-04-15,68.5,6.2,0.3
1992-05-15,68.7,6.2,0.3
...,...,...,...
2022-01-15,111.5,4.9,0.4
2022-02-15,112.5,5.1,0.9
2022-03-15,115.3,7.3,2.5


In [12]:
# normalizing dataframe to first entry
df = df/df.iloc[0]*100

# storing prepared dataframe
df.to_csv('preped_data/61111-0002_std.csv', sep=',')

df

,Verbraucherpreisindex,Veränderung_zum_Vorjahresmonat,Veränderung_zum_Vormonat
date,,,
1992-01-15,100.000000,100.000000,100.000000
1992-02-15,100.591716,103.571429,100.000000
1992-03-15,101.035503,110.714286,66.666667
1992-04-15,101.331361,110.714286,50.000000
1992-05-15,101.627219,110.714286,50.000000
...,...,...,...
2022-01-15,164.940828,87.500000,66.666667
2022-02-15,166.420118,91.071429,150.000000
2022-03-15,170.562130,130.357143,416.666667


In [13]:
# calculate year-over-year inflation rate
df_pct_2 = df.pct_change(freq=pd.DateOffset(years=1))
df_pct_2.dropna(inplace=True)

# calculate change in percent
df_pct_2 = df_pct_2*100

# storing edited dataframe
df_pct_2.to_csv('preped_data/61111-0002_pct.csv', sep=',')

df_pct_2

,Verbraucherpreisindex,Veränderung_zum_Vorjahresmonat,Veränderung_zum_Vormonat
date,,,
1993-01-15,4.585799,-17.857143,183.333333
1993-02-15,4.705882,-18.965517,16.666667
1993-03-15,4.538799,-27.419355,-25.000000
1993-04-15,4.525547,-27.419355,0.000000
1993-05-15,4.366812,-29.032258,-66.666667
...,...,...,...
2022-01-15,4.891816,390.000000,-50.000000
2022-02-15,5.140187,292.307692,28.571429
2022-03-15,7.255814,329.411765,400.000000
